In [ ]:
# Agent model library
import agentpy as ap

# Visualization
import matplotlib.pyplot as plt
import IPython

# Random
from random import randint

# Neighboring coordinates
NEIGHBOR_GRID = [
  (0, 1),    (1, 1),  (1, 0), (1, -1),
  (0, -1), (-1, -1), (-1, 0), (-1, 1),
]

In [ ]:
class CleaningModel(ap.Model):
  def setup(self):
    # Create agents (robots)
    n_robots = int(self.p.robots)
    robots = self.agents = ap.AgentList(self, n_robots)

    # Create agents (trash)
    n_trash = int(self.p.trash_density * self.p.width * self.p.height)
    trash = ap.AgentList(self, n_trash)
    self.agents.extend(trash)

    # Create grid (ground)
    self.ground = ap.Grid(self, (self.p.width, self.p.height), track_empty = True, check_border = True)
    self.ground.add_agents(robots, [(1,1)] * n_robots, empty = False)
    self.ground.add_agents(trash, random = True, empty = True)

    # Status (0: robot, 1: dirty, 2: clean)
    robots.status = 0
    trash.status = 1
    
  def step(self):
    # Stop if fully cleaned
    ground_trash = self.agents.select(self.agents.status == 1)
    if len(ground_trash) == 0:
      self.stop()

    # Iterate over every robot
    for robot in self.agents.select(self.agents.status == 0):
      # Get robot coordinates
      robot_x, robot_y = self.ground.positions[robot]
      will_move = True
      # Find trash at the robot's position
      for agent in self.ground.agents[robot_x, robot_y]:
        # Clean up trash
        if agent.status == 1:
          agent.status = 2
          will_move = False
          break
      # Move robot if it did not clean anything up
      if will_move:
        self.ground.move_by(robot, NEIGHBOR_GRID[randint(0, 7)])
    
  def end(self):
    collected_trash = len(self.agents.select(self.agents.status == 2))
    self.report('Percentage of trash collected',
    collected_trash / len(self.agents.status != 0))


In [ ]:
    
# Define parameters
parameters = {
'robots': 500, # Cleaning robots
'trash_density': 0.6, # Proportion of grid filled with trash
'width': 100, # Grid dimensions
'height': 100,
'steps': 500, # Simulation steps
}


In [ ]:
# Create single-run animation with custom colors
def animation_plot(model, ax):
  attr_grid = model.ground.attr_grid('status')
  color_dict = {0:'#585ded', 1:'#8c5858', 2:'#a1a1a1', None: "#ffffff"}
  ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
  ax.set_title(f"Simulation of a cleaning agent\n"
  f"Time-step: {model.t}, Trash left: "
  f"{len(model.agents.select(model.agents.status == 1))}")


In [ ]:
fig, ax = plt.subplots()
model = CleaningModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))